# 01_dart_scraper.ipynb

이 노트북에서는  
1. DART `corpCode.xml`을 내려받아 상장사 목록을 로드하고  
2. 각 사별로 list.json 페이징을 돌며 공시 메타정보를 수집한 뒤  
3. Selenium을 이용해 iframe HTML을 병렬로 가져와  
4. 최종 결과를 `dividend_with_text.csv` / `dividend_with_text.jsonl`으로 저장합니다.  

In [ ]:
import os
import time
import zipfile
from concurrent.futures import ThreadPoolExecutor, as_completed

import requests, chardet, xmltodict, json, pandas as pd
from requests.adapters import HTTPAdapter
from requests.exceptions import ReadTimeout
from urllib3.util.retry import Retry

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup
from tqdm import tqdm
from dotenv import load_dotenv

# .env에서 DART_API_KEY 로드
load_dotenv()
API_KEY = os.getenv("DART_API_KEY")
assert API_KEY, "❌ .env에 DART_API_KEY를 설정하세요"

# 저장 디렉토리
DATA_DIR = "data"
os.makedirs(DATA_DIR, exist_ok=True)

# User-Agent
HEADERS = {"User-Agent": "Mozilla/5.0"}

session = requests.Session()
retry_strategy = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[429,500,502,503,504],
    allowed_methods=["GET"]
)
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("https://", adapter)
session.mount("http://", adapter)

def _download_corp_code():
    zip_path = os.path.join(DATA_DIR, "corp_code.zip")
    xml_path = os.path.join(DATA_DIR, "corp_code.xml")
    # 기존 캐시 삭제
    for p in (zip_path, xml_path):
        if os.path.exists(p): os.remove(p)

    url = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
    r = session.get(url, headers=HEADERS, timeout=30); r.raise_for_status()
    content = r.content

    # XML directly?
    if content.lstrip().startswith(b"<?xml"):
        with open(xml_path, "wb") as f: f.write(content)
    else:
        with open(zip_path, "wb") as f: f.write(content)
        with zipfile.ZipFile(zip_path) as zf:
            zf.extractall(DATA_DIR)
        os.replace(os.path.join(DATA_DIR, "CORPCODE.xml"), xml_path)

def load_corps() -> pd.DataFrame:
    _download_corp_code()
    raw = open(os.path.join(DATA_DIR, "corp_code.xml"), "rb").read()
    enc = chardet.detect(raw)["encoding"]
    doc = xmltodict.parse(raw.decode(enc, errors="ignore"))

    # API 형태에 맞춰 list 뽑기
    if "result" in doc:
        lst = doc["result"].get("list", [])
    else:
        lst = doc["CORPCODE"]["list"]["item"]
    if isinstance(lst, dict): lst = [lst]

    df = pd.DataFrame(lst)[["corp_code","corp_name","stock_code"]]
    return df[df.stock_code.str.isdigit() & df.stock_code.str.len().eq(6)].reset_index(drop=True)

def list_filings(corp_code: str, bgn="20130101", end="20250630") -> list:
        all_list, fail = [], 0
        for page in range(1, 51):
            url = (
                f"https://opendart.fss.or.kr/api/list.json?"
                f"crtfc_key={API_KEY}&corp_code={corp_code}"
                f"&bgn_de={bgn}&end_de={end}"
                f"&page_count=100&page_no={page}"
            )
            try:
                r = session.get(url, headers=HEADERS, timeout=20)
                r.raise_for_status()
                page_list = r.json().get("list", [])
            except ReadTimeout:
                fail += 1
                time.sleep(1)
                if fail >= 3:
                    break
                continue
            except:
                break
    
            if not page_list:
                break
            all_list.extend(page_list)
            if len(page_list) < 100:
                break
            time.sleep(0.1)
        return all_list
    
    
def get_report_html(rcept_no: str, wait_sec=1.0) -> tuple:
        url = f"https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcept_no}"
        opts = Options()
        opts.add_argument("--headless")
        opts.add_argument("--disable-gpu")
        opts.add_argument(f"user-agent={HEADERS['User-Agent']}")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    
        try:
            driver.get(url)
            time.sleep(wait_sec)
            driver.switch_to.frame("ifrm")
            time.sleep(wait_sec)
            html = driver.page_source
        except:
            html = ""
        finally:
            driver.quit()
        return rcept_no, html

    # 1) 상장사 목록
df_corps = load_corps()

# 2) 모든 배당공시 메타 수집
tasks = []
for _, row in tqdm(df_corps.iterrows(), total=len(df_corps), desc="종목 처리"):
    for f in list_filings(row.corp_code):
        if "배당" in f["report_nm"]:
            tasks.append({
                "corp_name": row.corp_name,
                "stock_code":row.stock_code,
                "rcept_dt": f["rcept_dt"],
                "report_nm":f["report_nm"],
                "rcept_no":f["rcept_no"]
            })

# 3) HTML 병렬 스크래핑
results = []
with ThreadPoolExecutor(max_workers=5) as exe:
    futures = {exe.submit(get_report_html, t["rcept_no"]):t for t in tasks}
    for fut in tqdm(as_completed(futures), total=len(futures), desc="HTML 수집"):
        meta = futures[fut]; rno, html = fut.result()
        results.append({**meta, "html":html})

# 4) DataFrame으로 변환 & 저장
df = pd.DataFrame(results)
df.to_csv("data/dividend_with_text.csv", index=False, encoding="utf-8-sig")
with open("data/dividend_with_text.jsonl","w",encoding="utf-8") as fw:
    for rec in results:
        fw.write(json.dumps(rec, ensure_ascii=False)+"\n")

df = pd.read_csv("data/dividend_with_text.csv", encoding="utf-8-sig")
df.head()
df.shape

In [4]:
import pandas as pd
import FinanceDataReader as fdr

# 1. 파일 로드
csv_path = "/Users/gun/Desktop/미래에셋 AI 공모전/data/dividend_ml_ready.csv"
df = pd.read_csv(csv_path, encoding="utf-8-sig")

# 2. 상장 종목 리스트 가져오기 (KRX 기준)
krx_list = fdr.StockListing("KRX")
listed_codes = krx_list['Code'].astype(str).str.zfill(6).unique()

# 3. 필터링: 현재 상장된 종목만 남기기
df["stock_code"] = df["stock_code"].astype(str).str.zfill(6)
filtered_df = df[df["stock_code"].isin(listed_codes)].reset_index(drop=True)

# 4. 덮어쓰기 저장 (기존 파일 대체)
filtered_df.to_csv(csv_path, index=False, encoding="utf-8-sig")

# 5. 요약 출력
print(f"📄 원본 데이터: {len(df):,}건")
print(f"✅ 상장 종목 필터링 후: {len(filtered_df):,}건")

# ✅ 남은 고유 기업 수 확인
unique_firms = filtered_df["stock_code"].nunique()
print(f"🏢 남은 기업 수: {unique_firms:,}개")

print(f"💾 기존 파일 덮어쓰기 완료 → {csv_path}")

📄 원본 데이터: 15,996건
✅ 상장 종목 필터링 후: 15,460건
🏢 남은 기업 수: 1,682개
💾 기존 파일 덮어쓰기 완료 → /Users/gun/Desktop/미래에셋 AI 공모전/data/dividend_ml_ready.csv


In [5]:
import pandas as pd

# 1) ML용 CSV 로드
path = "/Users/gun/Desktop/미래에셋 AI 공모전/data/dividend_ml_ready.csv"
df = pd.read_csv(path, encoding="utf-8-sig")

# 2) per_share_common, yield_common, total_amount 중 하나라도 0인 경우
mask_any_zero = (
    (df.per_share_common == 0) |
    (df.yield_common      == 0) |
    (df.total_amount      == 0)
)

print(f"삭제 대상 행 수: {mask_any_zero.sum()} / 전체 {len(df)}")

# 3) 필터링
df_filtered = df[~mask_any_zero].reset_index(drop=True)

# 4) 원본 파일 덮어쓰기 저장
df_filtered.to_csv(path, index=False, encoding="utf-8-sig")
print(f"✅ 필터링 후 남은 행 수: {len(df_filtered)}. 덮어쓰기 완료: {path}")

삭제 대상 행 수: 0 / 전체 15460
✅ 필터링 후 남은 행 수: 15460. 덮어쓰기 완료: /Users/gun/Desktop/미래에셋 AI 공모전/data/dividend_ml_ready.csv
